# ETL Notebook – E-commerce Delivery Delay Dataset

This notebook implements the ETL (Extract, Transform, Load) process for the e-commerce delivery delay dataset.


In [67]:
import pandas as pd
import numpy as np
import os


## Load Data & Pre-cleaning Checks


In [68]:
print('Loading raw data...')
df = pd.read_csv('incom2024_delay_example_dataset.csv')

Loading raw data...


In [69]:
print('\n--- PRE-CLEANING CHECKS ---')
print('Shape:', df.shape)


--- PRE-CLEANING CHECKS ---
Shape: (15549, 41)


In [70]:
print('Missing values:')
print(df.isnull().sum())

Missing values:
payment_type                0
profit_per_order            0
sales_per_customer          0
category_id                 0
category_name               0
customer_city               0
customer_country            0
customer_id                 0
customer_segment            0
customer_state              0
customer_zipcode            0
department_id               0
department_name             0
latitude                    0
longitude                   0
market                      0
order_city                  0
order_country               0
order_customer_id           0
order_date                  0
order_id                    0
order_item_cardprod_id      0
order_item_discount         0
order_item_discount_rate    0
order_item_id               0
order_item_product_price    0
order_item_profit_ratio     0
order_item_quantity         0
sales                       0
order_item_total_amount     0
order_profit_per_order      0
order_region                0
order_state             

In [71]:
print('Duplicate rows:', df.duplicated().sum())

Duplicate rows: 0


In [72]:
print('\nData types:')
print(df.dtypes)


Data types:
payment_type                 object
profit_per_order            float64
sales_per_customer          float64
category_id                 float64
category_name                object
customer_city                object
customer_country             object
customer_id                 float64
customer_segment             object
customer_state               object
customer_zipcode            float64
department_id               float64
department_name              object
latitude                    float64
longitude                   float64
market                       object
order_city                   object
order_country                object
order_customer_id           float64
order_date                   object
order_id                    float64
order_item_cardprod_id      float64
order_item_discount         float64
order_item_discount_rate    float64
order_item_id               float64
order_item_product_price    float64
order_item_profit_ratio     float64
order_item_quan

## 2. Transform & Data Cleaning

### Remove duplicates

In [73]:
df = df.drop_duplicates()

### Standardize column names


In [74]:
df.columns = (
    df.columns
    .str.strip()
    .str.lower()
    .str.replace(' ', '_')
    .str.replace(r'[^\w_]', '', regex=True)
)


### Convert Data Types


In [75]:
print('\nConverting data types...')

date_cols = [col for col in df.columns if 'date' in col]
for col in date_cols:
    df[col] = pd.to_datetime(df[col], errors='coerce')

numeric_candidates = [
    'order_item_quantity',
    'order_item_product_price',
    'sales',
    'order_profit_per_order',
    'label'
]

for col in numeric_candidates:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')



Converting data types...


C:\Users\farah\AppData\Local\Temp\ipykernel_30824\968850659.py:5: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df[col] = pd.to_datetime(df[col], errors='coerce')
C:\Users\farah\AppData\Local\Temp\ipykernel_30824\968850659.py:5: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df[col] = pd.to_datetime(df[col], errors='coerce')


### Handle Duplicates & Missing Values


In [76]:
numeric_cols = df.select_dtypes(include=np.number).columns
categorical_cols = df.select_dtypes(include='object').columns

for col in numeric_cols:
    df[col] = df[col].fillna(df[col].median())

for col in categorical_cols:
    df[col] = df[col].fillna('unknown')


### Standardize Categorical Values


In [77]:
categorical_to_standardize = [
    'payment_type',
    'order_status',
    'customer_segment',
    'shipping_mode',
    'market'
]

for col in categorical_to_standardize:
    if col in df.columns:
        df[col] = (
            df[col]
            .str.lower()
            .str.strip()
            .str.replace(r'\s+', '_', regex=True)
            .str.replace(r'[^\w_]', '', regex=True)
        )

if 'market' in df.columns:
    df['market'] = df['market'].str.upper()


### standardize countries

In [78]:
country_mapping = {
    'ee. uu.': 'united_states',
    'ee. uu': 'united_states',
    'ee uu': 'united_states',
    'puerto rico': 'puerto_rico'
}

for col in df.columns:
    if 'country' in col:
        df[col] = df[col].str.lower().str.strip()
        for old, new in country_mapping.items():
            df[col] = df[col].str.replace(old, new, regex=False)


### Validate Label Column


In [79]:
if 'label' in df.columns:
    invalid_labels = df[~df['label'].isin([-1, 0, 1])]['label'].unique()

    if len(invalid_labels) > 0:
        print('Invalid label values found:', invalid_labels)
        df['label'] = df['label'].apply(
            lambda x: 1 if x > 0 else (-1 if x < 0 else 0)
        )
    else:
        print('Label values are valid (-1, 0, 1)')


Label values are valid (-1, 0, 1)


### Create derived columns


In [80]:
df['delivery_status'] = df['label'].map({
    -1: 'Early',
     0: 'On-time',
     1: 'Delayed'
})

df['order_date'] = pd.to_datetime(df['order_date'],utc=True , errors='coerce') 

if 'order_date' in df.columns:
    df['order_year'] = df['order_date'].dt.year
    df['order_month'] = df['order_date'].dt.month

df['is_delayed'] = np.where(df['label'] == 1, 1, 0)


## 3. Post-cleaning Checks & Load  

In [81]:
print('\n--- POST-CLEANING CHECKS ---')
print('Final shape:', df.shape)
print(df.isnull().sum())
print('\nDelivery status distribution:')
print(df['delivery_status'].value_counts())



--- POST-CLEANING CHECKS ---
Final shape: (15549, 45)
payment_type                0
profit_per_order            0
sales_per_customer          0
category_id                 0
category_name               0
customer_city               0
customer_country            0
customer_id                 0
customer_segment            0
customer_state              0
customer_zipcode            0
department_id               0
department_name             0
latitude                    0
longitude                   0
market                      0
order_city                  0
order_country               0
order_customer_id           0
order_date                  0
order_id                    0
order_item_cardprod_id      0
order_item_discount         0
order_item_discount_rate    0
order_item_id               0
order_item_product_price    0
order_item_profit_ratio     0
order_item_quantity         0
sales                       0
order_item_total_amount     0
order_profit_per_order      0
order_region   

In [ ]:

os.makedirs('data_cleaned', exist_ok=True)
df.to_csv('data_cleaned/cleaned_dataset.csv', index=False)
print('\n Cleaned data saved successfully')

PermissionError: [Errno 13] Permission denied: 'data_cleaned/cleaned_data.csv'